In [4]:
import mailparser
import re
import pandas as pd
import io
import glob

In [21]:
def parse_sitrep(path):
    text_df = pd.DataFrame() 
    for filename in glob.glob(path):
        mail = mailparser.parse_from_file(filename)
        text = mail.body
        a = text.split('\n\n')
        df = pd.DataFrame(a)
        df.columns =['test']
        df['A'], df['B'] = df['test'].str.split('-',1).str 
        df1 = df.transpose()
        df1.columns = df1.iloc[1]
        df2 = df1.reindex(df1.index.drop('A')).reset_index()
        df3 = df2.filter(regex='SITREP|^J|^L|^N.$').drop(0)
        df3 = df3.rename(columns={ df3.columns[0]: "cross_sitrep"})
        df3['cross_sitrep'] = df3.cross_sitrep.str.extract("CROSS (.*\/\d+/\d+)")
        text_df = text_df.append(df3, ignore_index=True)
    return text_df

    

In [22]:
parse_sitrep('./liste_sitrep/*.eml')

/Users/elsa.luneville/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


,J,L,N,cross_sitrep
0,PREMIERES MESURES PRISES\n1530 - CHEF DU PORT...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n01 NAVIRE DE ...,CORSEN/2017/1309
1,PREMIERES MESURES PRISES\n1124 - TEMOIN SIGNA...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n1 NAVIRE DE P...,CORSEN/2017/1335
2,PREMIERES MESURES PRISES\n0825 - INTERCEPTION...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n01 - UNE VEDE...,CORSEN/2017/1290
3,PREMIERES MESURES PRISES\n1045 - PROPRIETAIRE...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n1 NAVIRE DE P...,CORSEN/2017/1327
4,PREMIERES MESURES PRISES\n0755 - SEMPAHORE DE...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n01 EMBARCATIO...,CORSEN/2017/1285
5,PREMIERES MESURES PRISES\n1343 - NAVIRE DE PL...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n01 NAVIRE DE ...,CORSEN/2017/1237
6,PREMIERES MESURES PRISES\n1408 - SEMAPHORE DE...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n01 NAVIRE DE ...,CORSEN/2017/1319
7,"PREMIERES MESURES PRISES\n1530 - VEDETTE ""EVA...",CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n1 EMBARCATION...,CORSEN/2017/1292
8,PREMIERES MESURES PRISES\n0814 - SNSM LOCQU...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\n1 NAVIRE DE P...,CORSEN/2017/1251
9,PREMIERES MESURES PRISES\n1003 - CODIS 22 REL...,CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION\n...,RENSEIGNEMENTS COMPLEMENTAIRES\nNEANT,CORSEN/2017/1330


In [23]:
        mail = mailparser.parse_from_file('./liste_sitrep/SITREP SAR UN  ET FINAL - - CROSS CORSEN-2017-1272.eml')
        text = mail.body
        a = text.split('\n\n')
        df = pd.DataFrame(a)
        df.columns =['test']
        df['A'], df['B'] = df['test'].str.split('\n',1).str 
        df1 = df.transpose()
        df1.columns = df1.iloc[1]
        df2 = df1.reindex(df1.index.drop('A')).reset_index()
        df3 = df2.filter(regex='SITREP|^J|^L|^N.$').drop(0)
        df3 = df3.rename(columns={ df3.columns[0]: "cross_sitrep"})
        df3['cross_sitrep'] = df3.cross_sitrep.str.extract("CROSS (.*\/\d+/\d+)")

/Users/elsa.luneville/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if sys.path[0] == '':


In [28]:
df2

A,index,DE CROSS CORSEN,SITREP SAR UN ET FINAL - - CROSS CORSEN/2017/1272,A - IDENTITE VEHICULE(S) IMPLIQUE(S),B - POSITION,C - SITUATION,D - PERSONNES,E - ASSISTANCE DEMANDEE,F - CENTRE DE COORDINATION,G - DESCRIPTION VEHICULE(S) IMPLIQUE(S),H - METEO SUR ZONE,J - PREMIERES MESURES PRISES,K - ZONES DE RECHERCHE,L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION,M - PLANS FUTURS,N - RENSEIGNEMENTS COMPLEMENTAIRES
0,test,DE CROSS CORSEN\nA PREMAR ATLANT AEM\nSGMER\nP...,SITREP SAR UN ET FINAL - - CROSS CORSEN/2017/...,A - IDENTITE VEHICULE(S) IMPLIQUE(S)\nNOM : SM...,"B - POSITION\n48° 39,17' N / 002° 02,25' O\nAZ...",C - SITUATION\nALERTE LE : 26NOV2017 1030 / PA...,D - PERSONNES\n1 PERSONNE(S) IMPLIQUEE(S)\nBIL...,E - ASSISTANCE DEMANDEE\nRECHERCHE ET SAUVETAGE,F - CENTRE DE COORDINATION\nCROSS CORSEN,G - DESCRIPTION VEHICULE(S) IMPLIQUE(S)\nTYPE ...,H - METEO SUR ZONE\nORIGINE : SEMAPHORE ST CAS...,J - PREMIERES MESURES PRISES\n1030 - INTERCEPT...,K - ZONES DE RECHERCHE\nNEANT,L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATIO...,M - PLANS FUTURS\nNEANT,N - RENSEIGNEMENTS COMPLEMENTAIRES\n01 NAVIRE ...
1,B,A PREMAR ATLANT AEM\nSGMER\nPREMAR ATLANT\nCOF...,TOUTES HEURES UTC,NOM : SM 1 / PAVILLON : FRANCE\nIMMAT : 000000,"48° 39,17' N / 002° 02,25' O\nAZIMUT/DISTANCE ...",ALERTE LE : 26NOV2017 1030 / PAR : COMITE DE C...,1 PERSONNE(S) IMPLIQUEE(S)\nBILAN : 1 SECOURUE(S),RECHERCHE ET SAUVETAGE,CROSS CORSEN,"TYPE : PLAISANCE A VOILE\nLONG (M) : 8,0",ORIGINE : SEMAPHORE ST CAST / ST MALO / GDH : ...,1030 - INTERCEPTION ECHANGE VHF ENTRE COMITE C...,NEANT,"1036 - EQUIPIERE RECUPEREE PAR VOILIER ""IRITI...",NEANT,01 NAVIRE DE PLAISANCE A VOILE ASSISTE\nFAIBLE...
